# Bachelor's thesis topic: Sentiment analysis based on multifractal methods

Sentiment analysis is crucial for decision-making in corporate platforms and for users. Traditional techniques face limitations, such as low prediction accuracy, large training datasets, and computational inefficiency. My project addresses these limitations by applying multifractal methods combined with machine learning techniques, reducing data features while preserving accuracy.








# Data Loading

The RuReviews corpus consisting of approximately 19,000 user reviews of the online women's clothing store was used as data.

In [58]:
import numpy as np
import pandas as pd

full_data=pd.read_csv('review_input.csv',sep=',')

In [59]:
full_data

,Original_review,Sentiment
0,Шапочка очень хорошая. Доставка быстрая. Спаси...,1
1,"заказ не пришел, деньги не вернули, спор закры...",0
2,очень долго ждала заказ. Так и не пришел. откр...,0
3,Очень удобные для йоги. Цвет как на фото,1
4,Очень долго думала брать шарф или нет и все та...,0
...,...,...
19063,"спасибо продавцу! рубашка в хорошем состоянии,...",1
19064,Покупала в распродажу. В первые минуты осталос...,1
19065,"Яркие, милые и качество на высоте!",1
19066,"Товар так и не пришёл, спор не открывала. С пр...",0


## 1. Data preprocessing

In [60]:
import nltk
!pip install pymystem3
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')

import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import string
import re
from pymystem3 import Mystem

def remove_stopwords(i):
    stop_words = stopwords.words('russian')
    stop_words.remove('не')
    stop_words.remove('нет')
    filtered_sentence = []
    for w in i:
        if w not in stop_words:
            filtered_sentence.append(w)
    return filtered_sentence

def data_preprocessing(data):
    num_steps = 12
    step = 1
    print('Step {}/{}: Drop duplicats and empty sentences '.format(step, num_steps))
    data.drop_duplicates(subset='Original_review',keep='first', inplace=True,)
    data.dropna(inplace=True)
    data=data.reset_index(drop=True)
    step += 1
    print('Step {}/{}: Converting to lower case'.format(step, num_steps))
    data['Review']= [i.lower() for i in data['Original_review']]

    step += 1
    print('Step {}/{}: Removing punctuation'.format(step, num_steps))
    data['Review'] = [ re.sub(r'[^\w\s]', ' ', i) for i in data['Review']]

    step += 1
    print('Step {}/{}: Removing single words'.format(step, num_steps))
    data['Review'] = [re.sub(r"\b[а-яА-Я]\b", " ", i) for i in data['Review']]

    step += 1
    print('Step {}/{}: Removing mentions and hashtag symbols'.format(step, num_steps))
    data['Review'] = [re.sub(r"@[a-zA-Z0-9]+|\#[a-zA-Z0-9]", " ", str(i)) for i in data['Review']]

    step += 1
    print('Step {}/{}: Removing URLs'.format(step, num_steps))
    data['Review'] = [re.sub(r'http\S+', " ", str(i)) for i in data['Review']]

    step += 1
    print('Step {}/{}: Removing numbers'.format(step, num_steps))
    data['Review'] = [re.sub(r"[0-9]", " ", i) for i in data['Review']]

    step += 1
    print('Step {}/{}: Removing latin letters'.format(step, num_steps))
    data['Review'] = [re.sub(r"[a-zA-Z]", " ", i) for i in data['Review']]

    step += 1
    print('Step {}/{}: Removing empty space'.format(step, num_steps))
    data['Review'] = [' '.join(i.split())for i in data['Review']]

    step += 1
    print('Step {}/{}: Lemmatization'.format(step, num_steps))
    mystem = Mystem()
    data['Review'] = [mystem.lemmatize(i) for i in data['Review']]

    step += 1
    print('Step {}/{}: Removing stop words'.format(step, num_steps))
    data['Review'] = [remove_stopwords(i) for i in data['Review']]
    step += 1

    print('Step {}/{}: Removing empty sentences'.format(step, num_steps))
    data = data[(data['Review'].str.len() != 0)]
    data['Review']=[' '.join(i) for i in data['Review']]
    data['Review'] = [' '.join(i.split())for i in data['Review']]
    data['Review'] = [i.replace("\n","") for i in data['Review']]
    data.drop_duplicates(subset='Review',keep='first',inplace=True)
    data['Review'] = [i.split() for i in data['Review']]
    return data


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [148]:
prepr_data=data_preprocessing(full_data)

Step 1/12: Drop duplicats and empty sentences 
Step 2/12: Converting to lower case
Step 3/12: Removing punctuation
Step 4/12: Removing single words
Step 5/12: Removing mentions and hashtag symbols
Step 6/12: Removing URLs
Step 7/12: Removing numbers
Step 8/12: Removing latin letters
Step 9/12: Removing empty space
Step 10/12: Lemmatization
Step 11/12: Removing stop words
Step 12/12: Removing empty sentences


In [126]:
prepr_data

,Original_review,Sentiment,Review
0,Шапочка очень хорошая. Доставка быстрая. Спаси...,1,"[шапочка, очень, хороший, доставка, быстрый, с..."
1,"заказ не пришел, деньги не вернули, спор закры...",0,"[заказ, не, приходить, деньги, не, вернуть, сп..."
2,очень долго ждала заказ. Так и не пришел. откр...,0,"[очень, долго, ждать, заказ, не, приходить, от..."
3,Очень удобные для йоги. Цвет как на фото,1,"[очень, удобный, йога, цвет, фото]"
4,Очень долго думала брать шарф или нет и все та...,0,"[очень, долго, думать, брать, шарф, нет, весь,..."
...,...,...,...
19063,"спасибо продавцу! рубашка в хорошем состоянии,...",1,"[спасибо, продавец, рубашка, хороший, состояни..."
19064,Покупала в распродажу. В первые минуты осталос...,1,"[покупать, распродажа, первый, минута, остават..."
19065,"Яркие, милые и качество на высоте!",1,"[яркий, милый, качество, высота]"
19066,"Товар так и не пришёл, спор не открывала. С пр...",0,"[товар, не, приходить, спор, не, открывать, пр..."


Choose only 5000 positive and negative comments randomly

In [149]:
from sklearn.utils import shuffle

prepr_data = shuffle(prepr_data)
positive=prepr_data[prepr_data.Sentiment==1]
negative=prepr_data[prepr_data.Sentiment==0]
min_value=5000
prepr_data =pd.concat([positive[:min_value],negative[:min_value]])
prepr_data = shuffle(prepr_data )
data=prepr_data.reset_index(drop=True)

In [150]:
data

,Original_review,Sentiment,Review
0,"Товар так и не пришёл , был отправлен на невер...",0,"[товар, не, приходить, отправлять, неверный, а..."
1,"Отправил не тот товар, Да еще и деньги не возв...",0,"[отправлять, не, товар, деньги, не, возвращать]"
2,Заказываю 2- ой раз. в первый раз заказала бод...,0,"[заказывать, ой, первый, заказывать, боди, гор..."
3,Я очень недовольна продавцом и обслуживанием! ...,0,"[очень, недовольный, продавец, обслуживание, в..."
4,"Заказ пришел быстро,но у меня на трусы началас...",0,"[заказ, приходить, быстро, трусы, начинаться, ..."
...,...,...,...
9995,"до Красноярского края 2,5 месяца.Размер заказы...",0,"[красноярский, край, месяц, размер, заказывать..."
9996,Две недели до Казани! Хорошая рубашка,1,"[неделя, казань, хороший, рубашка]"
9997,приятная ткань,1,"[приятный, ткань]"
9998,Отправили не того цвета. Я заказывала винный ц...,0,"[отправлять, не, цвет, заказывать, винный, цве..."


## 3. Words embedding

Navec is a library of pretrained word embeddings for russian language.

In [10]:
!pip install navec

In [11]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

--2024-01-11 17:35:58--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50.56M  16.1MB/s    in 3.1s    

2024-01-11 17:36:02 (16.1 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ saved [53012480/53012480]



In [151]:
from navec import Navec

path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)
def emb(line):
    embedding=[navec[i] for i in line if i in navec]
    return embedding

embedding=[emb(i) for i in data.Review]

for i in range(len(embedding)):
  if embedding[i]==[]:
    print(i)
    embedding=embedding.pop(i)
    data=data.drop(labels=[i],axis = 0)

data=data.reset_index(drop=True)


In [130]:
data

,Original_review,Sentiment,Review
0,"очаровательная блузка, нежная, аккуратная, спа...",1,"[очаровательный, блузка, нежный, аккуратный, с..."
1,"Поступили некрасиво,продавец толком не общался...",0,"[поступать, некрасиво, продавец, толком, не, о..."
2,"Юбка отличная! \r\nКачество на высоте, xxl на ...",1,"[юбка, отличный, качество, высота, подходить, ..."
3,"Хорошие носочки, доставка быстрая, меньше меся...",1,"[хороший, носочек, доставка, быстрый, мало, ме..."
4,Спасибо продавцу пришла быстро))),1,"[спасибо, продавец, приходить, быстро]"
...,...,...,...
3995,заказ не пришёл. трек не двигался.,0,"[заказ, не, приходить, трек, не, двигаться]"
3996,"Товар не соответствует указанному, тонкая синт...",0,"[товар, не, соответствовать, указывать, тонкий..."
3997,"Свитер отличный. Качеств хорошее, приятно к те...",1,"[свитер, отличный, качество, хороший, приятно,..."
3998,Я в восторге от этой куртки!!!Мало того что це...,1,"[восторг, куртка, мало, цена, приемлемый, каче..."


## 4. Sentences embedding

By averaging embedding vectors in a sentence, one can obtain a vector representation of the sentence that reflects its meaning.

In [152]:
emb_mean=[]
emb_sent=[]
for i in range(len(embedding)):
  sum= np.zeros(len(embedding[i][0]))
  for j in  range(len(embedding[i])):
    sum=sum+embedding[i][j]
  sum/=len(embedding[i])
  emb_sent.append(sum)


In [153]:
data['Embedding']=emb_sent

In [116]:
data

,Original_review,Sentiment,Review,Embedding
0,"Эта кофта настолько маленькая, что налезете то...",0,"[кофта, настолько, маленький, налезать, ребено...","[-0.0868458187000619, -0.15847761225369242, 0...."
1,"Качество дешевенькое. Материал непонятный, нап...",0,"[качество, дешевенький, материал, непонятный, ...","[-0.10457761083833045, -0.15687840926047, 0.07..."
2,"пуховик замечательный, быстрая доставка, трек ...",1,"[пуховик, замечательный, быстрый, доставка, тр...","[-0.02281218022108078, 0.04642141034657305, -0..."
3,Длина размера L 71 см. Не надо обманывать. Мо...,0,"[длина, размер, см, не, обманывать, открывать,...","[0.07541657700425103, -0.24144409738835834, 0...."
4,Спасибо за супербыструю доставку. Заказала 16....,1,"[спасибо, супербыстрый, доставка, заказывать, ...","[-0.11635113234321276, -0.13621194045990706, 0..."
...,...,...,...,...
3995,"Пришли с браком и вообще модель дурацкая, сраз...",0,"[приходить, брак, вообще, модель, дурацкий, ср...","[0.13146982395223208, -0.2447807903933738, 0.0..."
3996,Шикарно смотрится и сидит. Материал полиэстер ...,1,"[шикарно, смотреться, сидеть, материал, полиэс...","[0.16268630201617876, -0.17911740796019635, 0...."
3997,"Товар не пришёл , пришлось ОЧЕНЬ долго спорить...",0,"[товар, не, приходить, приходиться, очень, дол...","[-0.05491637780020634, -0.2661188058555126, 0...."
3998,Одна майка меньше другой((( получается у одной...,0,"[майка, маленький, получаться, не, очень, комф...","[0.019282352722560365, -0.1506500281393528, 0...."


# 5. Multifractal Detrended Fluctuation Analysis

Implementation of MF-DFA approach from scratch

In [96]:
!pip install MFDFA
from MFDFA import MFDFA
import math
import matplotlib.pyplot as plt
import math

In [154]:
s = np.arange(5,25,3)
order =3
result_dfa=[]
index=0
for word in emb_sent:
  h=[]
  q_=[1,2,3,4,5,6]
  for q in q_:
    y=[]
    lag, dfa = MFDFA(np.array(word), lag = s, q = q, order = order)
    dfa=dfa.flatten()
    dfa=[math.log(i) for i in dfa]
    y.append(dfa)
    x = [math.log(i) for i in lag]
    p1 = np.poly1d(np.polyfit(x, dfa, 1))
    h.append(p1.c[0])
  result_dfa.append(h)
  index+=1

In [155]:
data['MF_DFA']=result_dfa

# Random Forest classification based on MF-DFA approach

In [174]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data.MF_DFA, data.Sentiment, test_size=0.4, random_state=42, shuffle=True)

In [175]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import tensorflow as tf
model = RandomForestClassifier()
print(len(X_train), len(X_test), len(y_train), len(y_test))
X_train=tf.keras.preprocessing.sequence.pad_sequences(X_train,dtype='float64',padding='post')
X_test=tf.keras.preprocessing.sequence.pad_sequences(X_test,dtype='float64',padding='post')
model.fit(X_train, y_train)
prediction_linear_rf = model.predict(X_test)
report = classification_report(y_test, prediction_linear_rf)
print(report)


6000 4000 6000 4000
              precision    recall  f1-score   support

           0       0.95      0.92      0.93      2014
           1       0.92      0.95      0.93      1986

    accuracy                           0.93      4000
   macro avg       0.93      0.93      0.93      4000
weighted avg       0.93      0.93      0.93      4000



# 5.  Multifractal detrended moving average analysis

Implementation of MF-DMA approach from scratch (Applying this approach might take a long time)

In [160]:
import matplotlib.pyplot as plt
import math

def mf_dma(word,s=7, q=-2):
  y=[]
  sum=0
  for i in word:
    sum+=i
    y.append(sum)
  y_=[]
  E=[]
  for i in range(s,len(y)+1):
    sum=np.sum(y[i-s:i])
    temp=y[i-1]-sum/len(y[i-s:i])
    y_.append(sum/s)
    E.append(temp)
  I_s=math.floor(len(y_)/s)
  F_2=[]
  for i in range(s,len(E)+1,s):
    pow=np.power(E[i-s:i],2)
    sum=np.sum(pow)
    F_2.append(sum/s)
  for i in range(len(E),s-1,-s):
    pow=np.power(np.flip(E[i-s:i],0),2)
    sum=np.sum(pow)
    F_2.append(sum/s)
  if q!=0:
    sum=0
    for i in range(len(F_2)):
      pow=math.pow(F_2[i],(q/2))
      sum=sum+pow
    sum/=2*I_s
    F_q=math.pow(sum,(1/q))
  else:
    sum=0
    for i in range(len(F_2)):
      sum+=math.log(F_2[i])
    sum/=4*I_s
    F_q=math.exp(sum)
  return math.log(F_q)

In [161]:
def hurst(word, q=-10):
  s = np.arange(3,13,1)
  x = [math.log(i) for i in s]
  y = [mf_dma(word,i,q) for i in s]
  p1 = np.poly1d(np.polyfit(x, y, 1))
  h.append(p1.c[0])

In [162]:
result_dma=[]
index=0

for i in data.Embedding:
  h=[]
  q_=[0,1,2,3,4,5]
  for q in q_:
    hurst(i, q)
  result_dma.append(h)
  index+=1

In [163]:
data['MF_DMA']=result_dma

# Random Forest classification based on MF-DMA approach

In [172]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data.MF_DMA, data.Sentiment, test_size=0.4, random_state=42, shuffle=True)

In [173]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import tensorflow as tf
model = RandomForestClassifier()
print(len(X_train), len(X_test), len(y_train), len(y_test))
X_train=tf.keras.preprocessing.sequence.pad_sequences(X_train,dtype='float64',padding='post')
X_test=tf.keras.preprocessing.sequence.pad_sequences(X_test,dtype='float64',padding='post')
model.fit(X_train, y_train)
prediction_linear_rf = model.predict(X_test)
report = classification_report(y_test, prediction_linear_rf)
print(report)


6000 4000 6000 4000
              precision    recall  f1-score   support

           0       0.95      0.91      0.93      2014
           1       0.92      0.95      0.93      1986

    accuracy                           0.93      4000
   macro avg       0.93      0.93      0.93      4000
weighted avg       0.93      0.93      0.93      4000



# Random Forest classification based on classical approach based on GloVe embedding

In [170]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data.Embedding, data.Sentiment, test_size=0.4, random_state=42, shuffle=True)

In [171]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import tensorflow as tf
model = RandomForestClassifier()
print(len(X_train), len(X_test), len(y_train), len(y_test))
X_train=tf.keras.preprocessing.sequence.pad_sequences(X_train,dtype='float64',padding='post')
X_test=tf.keras.preprocessing.sequence.pad_sequences(X_test,dtype='float64',padding='post')
model.fit(X_train, y_train)
prediction_linear_rf = model.predict(X_test)
report = classification_report(y_test, prediction_linear_rf)
print(report)


6000 4000 6000 4000
              precision    recall  f1-score   support

           0       0.95      0.87      0.91      2014
           1       0.88      0.96      0.91      1986

    accuracy                           0.91      4000
   macro avg       0.91      0.91      0.91      4000
weighted avg       0.91      0.91      0.91      4000

